# 🌌 Halo Formation

#### Pragun Nepal  
#### June 21, 2025

---

This notebook contains the code for running the Halo Formation Simulation (with and without axes). The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the coupled N-body, FoF-Halo-Finder, and ReionYuga Repositories by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Setting up ffmpeg for Jupyter Lab](#Setting_up_ffmpeg_for_Jupyter_Lab)
2. [Code for Halo Formation Simulation](#Code_for_N-body_Simulation)
3. [Code for Halo Formation Simulation (with axes)](#Code_for_N-body_Simulation_axes)
4. [Conclusion](#conclusion)

---

## 🔭 <a name="Setting_up_ffmpeg_for_Jupyter_Lab"></a>1. Setting up ffmpeg for Jupyter Lab



```python
import os
import matplotlib as mpl

#Enter the Full Fath of the ffmeg library - It may be different for you
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

#Prepend its directory to PATH for this session
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ["PATH"]

#Tell Matplotlib exactly where to find it
mpl.rcParams["animation.ffmpeg_path"] = ffmpeg_path

#Verification
import shutil
print("→ PATH is now:", os.environ["PATH"].split(os.pathsep)[:3], "…")
print("→ which(ffmpeg):", shutil.which("ffmpeg"))
print("→ rcParams ffmpeg_path:", mpl.rcParams["animation.ffmpeg_path"])
```



If ffmpeg is not installed in your PC. You can do so by visiting the website - https://ffmpeg.org/download.html



---

## ⚙️ <a name="Code_for_N-body_Simulation"></a>2. Code for Halo Formation Simulation (without axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 registers 3D projection
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.spatial import cKDTree

#Extract redshift values from halo filename

def extract_redshift(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1])

#Load & sort halo data

data_path = "/Users/pragunnepal/Downloads/500Halos"
#you can enter the path for your data files here
files = sorted(
    glob.glob(os.path.join(data_path, "halo_*.txt")),
    key=extract_redshift,
    reverse=True
)

raw_frames = []
redshifts = []
for f in files:
    z = extract_redshift(f)
    try:
        data = np.loadtxt(f).reshape(-1, 4)
    except Exception:
        data = np.zeros((0, 4))
    raw_frames.append(data)
    redshifts.append(z)

#True frame-to-frame interpolation to reduce flicker
n_interp = 4  # number of in-between frames; adjust for smoothness
frames_data, frames_z = [], []

def interp_pair(A, B, zA, zB, steps):
    #if one frame is empty, hold the other
    if A.shape[0] == 0 or B.shape[0] == 0:
        for t in np.linspace(0, 1, steps + 1):
            frames_data.append(A if B.shape[0] == 0 else B)
            frames_z.append(zA * (1 - t) + zB * t)
        return

    #match halos by nearest neighbor in 3D
    tree = cKDTree(A[:, 1:])
    _, idx = tree.query(B[:, 1:], k=1)
    A_matched = A[idx]

    #interpolate mass+pos arrays
    for t in np.linspace(0, 1, steps + 1):
        interp = A_matched * (1 - t) + B * t
        frames_data.append(interp)
        frames_z.append(zA * (1 - t) + zB * t)

#build interpolated sequence
for (A, zA), (B, zB) in zip(zip(raw_frames[:-1], redshifts[:-1]), zip(raw_frames[1:], redshifts[1:])):
    interp_pair(A, B, zA, zB, n_interp)
#append last raw frame
frames_data.append(raw_frames[-1])
frames_z.append(redshifts[-1])

#Animation parameters

fps = 30
rotation_speed = 0.1
extra_hold = fps * 33
zoom_in, zoom_out = fps * 15, fps * 5
max_zoom = 1.2

total_motion = len(frames_data)
total_frames = total_motion + extra_hold

#Compute bounds & centers

sample_data = [f[:, 1:] for f in raw_frames if f.size > 0]
if sample_data:
    samples = np.vstack(sample_data)
else:
    samples = np.zeros((1, 3))
bounds = [(samples[:, i].min(), samples[:, i].max()) for i in range(3)]
bounds = [(mn - 0.2*(mx-mn), mx + 0.2*(mx-mn)) for mn, mx in bounds]
centers = [0.5*(mn+mx) for mn, mx in bounds]

#Setup figure & axes

dpi = 300
fig = plt.figure(figsize=(6,6), dpi=dpi)
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black'); fig.patch.set_facecolor('black')
ax.axis('off'); ax.set_box_aspect([1,1,1])
elev0, azim0 = 25, 30
ax.view_init(elev0, azim0)
scat = ax.scatter([], [], [], s=[], c='gold', alpha=0.7, edgecolors='none')
txt = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='white')
ax_triad = inset_axes(ax, width="20%", height="20%", loc='lower left')
ax_triad.axis('off')

#rotation helper

def rotation_matrix(ang_deg):
    th = np.deg2rad(ang_deg)
    Rz = np.array([[np.cos(th), -np.sin(th), 0], [np.sin(th), np.cos(th), 0], [0,0,1]])
    Rx = np.array([[1,0,0], [0,np.cos(th), -np.sin(th)], [0, np.sin(th), np.cos(th)]])
    return Rz @ Rx

#update function

def update(idx):
    if idx < total_motion:
        data, zval = frames_data[idx], frames_z[idx]
    else:
        data, zval = frames_data[-1], frames_z[-1]

    if data.size == 0:
        scat._offsets3d = ([], [], [])
        scat.set_sizes([])
    else:
        masses = data[:, 0]
        coords = data[:, 1:]
        radii = np.cbrt(masses)
        sizes = (radii / radii.max()) * 50
        scat._offsets3d = (coords[:,0], coords[:,1], coords[:,2])
        scat.set_sizes(sizes)

    txt.set_text(f'z = {zval:.2f}')
    angle = idx * rotation_speed
    ax.view_init(elev0 + angle, azim0 + angle)

    #triad inset
    ax_triad.clear(); ax_triad.axis('off')
    R = rotation_matrix(angle)
    for col, (lbl, vec) in zip(['red','green','blue'], [('X', np.array([1,0,0])), ('Y', np.array([0,1,0])), ('Z', np.array([0,0,1]))]):
        v = R @ vec[1] if False else R @ vec
        end = np.array([0.5,0.5]) + 0.3 * v[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5,0.5), arrowprops=dict(arrowstyle='->', color=col))
        ax_triad.text(end[0], end[1], vec[0] if False else lbl, color=col, fontsize=8)

    #zoom logic
    if idx >= total_motion:
        h = idx - total_motion
        if h <= zoom_in:
            f = 1 + (max_zoom-1)*(h/zoom_in)
        else:
            ho = h-zoom_in
            f = max_zoom - (max_zoom-1)*min(ho/zoom_out, 1)
    else:
        f = 1

    for i, (mn, mx) in enumerate(bounds):
        c = centers[i]; r = (mx-mn)/2
        low, high = c - r/f, c + r/f
        if i==0: ax.set_xlim(low, high)
        elif i==1: ax.set_ylim(low, high)
        else: ax.set_zlim(low, high)

    return scat, txt

#Build & save animation

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000/fps, blit=False)
writer = FFMpegWriter(fps=fps, metadata={'artist':'PragunNepal'}, bitrate=30000)
ani.save('HaloFormation.mp4', writer=writer)
print('✅ Saved HaloFormation.mp4')
```
---

## ⚙️ <a name="Code_for_N-body_Simulation_axes"></a>3. Code for Halo Formation (with axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D  # registers 3D projection
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.spatial import cKDTree

#Extract redshift from halo filename
def extract_redshift(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1])

#Load & sort halo data
data_path = "/Users/pragunnepal/Downloads/500Halos"
#you can enter the path for your data files here
files = sorted(
    glob.glob(os.path.join(data_path, "halo_*.txt")),
    key=extract_redshift,
    reverse=True
)

raw_frames = []
redshifts = []
for f in files:
    z = extract_redshift(f)
    try:
        data = np.loadtxt(f).reshape(-1, 4)
    except Exception:
        data = np.zeros((0, 4))
    raw_frames.append(data)
    redshifts.append(z)

#True frame-to-frame interpolation to reduce flicker
n_interp = 4  # number of in-between frames; adjust for smoothness
frames_data, frames_z = [], []

def interp_pair(A, B, zA, zB, steps):
    # if one frame is empty, hold the other
    if A.shape[0] == 0 or B.shape[0] == 0:
        for t in np.linspace(0, 1, steps + 1):
            frames_data.append(A if B.shape[0] == 0 else B)
            frames_z.append(zA * (1 - t) + zB * t)
        return

    #match halos by nearest neighbor in 3D
    tree = cKDTree(A[:, 1:])
    _, idx = tree.query(B[:, 1:], k=1)
    A_matched = A[idx]

    #interpolate mass+pos arrays
    for t in np.linspace(0, 1, steps + 1):
        interp = A_matched * (1 - t) + B * t
        frames_data.append(interp)
        frames_z.append(zA * (1 - t) + zB * t)

#build interpolated sequence

for (A, zA), (B, zB) in zip(zip(raw_frames[:-1], redshifts[:-1]), zip(raw_frames[1:], redshifts[1:])):
    interp_pair(A, B, zA, zB, n_interp)
#append last raw frame
frames_data.append(raw_frames[-1])
frames_z.append(redshifts[-1])

#Animation parameters

fps = 30
rotation_speed = 0.1
extra_hold = fps * 33
zoom_in, zoom_out = fps * 15, fps * 5
max_zoom = 1.2

total_motion = len(frames_data)
total_frames = total_motion + extra_hold

#Compute bounds & centers

sample_data = [f[:, 1:] for f in raw_frames if f.size > 0]
if sample_data:
    samples = np.vstack(sample_data)
else:
    samples = np.zeros((1, 3))
bounds = [(samples[:, i].min(), samples[:, i].max()) for i in range(3)]
bounds = [(mn - 0.2*(mx-mn), mx + 0.2*(mx-mn)) for mn, mx in bounds]
centers = [0.5*(mn+mx) for mn, mx in bounds]

#Setup figure & axes

dpi = 300
fig = plt.figure(figsize=(6,6), dpi=dpi)
ax = fig.add_subplot(111, projection='3d')

#background black

ax.set_facecolor('black'); fig.patch.set_facecolor('black')

#configure 3D grid on each axis with nearly invisible visibility

for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    axis._axinfo['grid']['color'] = 'purple'
    axis._axinfo['grid']['linewidth'] = 0.10
    axis._axinfo['grid']['alpha'] = 0.001  # almost invisible

#axis panes extremely subtle purple

for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    axis.pane.set_facecolor('purple')
    axis.pane.set_edgecolor('purple')
    axis.pane.set_alpha(0.001)

#faint purple ticks

ax.tick_params(colors='purple', which='both', labelsize=6)
ax.set_box_aspect([1,1,1])

#turn axes on but subtle

ax.axis('on')

elev0, azim0 = 25, 30
ax.view_init(elev0, azim0)
scat = ax.scatter([], [], [], s=[], c='gold', alpha=0.7, edgecolors='none')
txt = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='white')
ax_triad = inset_axes(ax, width="20%", height="20%", loc='lower left')
ax_triad.axis('off')

#rotation helper

def rotation_matrix(ang_deg):
    th = np.deg2rad(ang_deg)
    Rz = np.array([[np.cos(th), -np.sin(th), 0], [np.sin(th), np.cos(th), 0], [0,0,1]])
    Rx = np.array([[1,0,0], [0,np.cos(th), -np.sin(th)], [0, np.sin(th), np.cos(th)]])
    return Rz @ Rx

#update function

def update(idx):
    if idx < total_motion:
        data, zval = frames_data[idx], frames_z[idx]
    else:
        data, zval = frames_data[-1], frames_z[-1]

    if data.size == 0:
        scat._offsets3d = ([], [], [])
        scat.set_sizes([])
    else:
        masses = data[:, 0]
        coords = data[:, 1:]
        radii = np.cbrt(masses)
        sizes = (radii / radii.max()) * 50
        scat._offsets3d = (coords[:,0], coords[:,1], coords[:,2])
        scat.set_sizes(sizes)

    txt.set_text(f'z = {zval:.2f}')
    angle = idx * rotation_speed
    ax.view_init(elev0 + angle, azim0 + angle)

    #triad inset
    ax_triad.clear(); ax_triad.axis('off')
    R = rotation_matrix(angle)
    axes = {'X': np.array([1,0,0]), 'Y': np.array([0,1,0]), 'Z': np.array([0,0,1])}
    for color, (lbl, vec) in zip(['red','green','blue'], axes.items()):
        v = R @ vec
        end = np.array([0.5,0.5]) + 0.3 * v[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5,0.5), arrowprops=dict(arrowstyle='->', color=color, alpha=0.6))
        ax_triad.text(end[0], end[1], lbl, color=color, fontsize=8, alpha=0.6)

    #zoom logic
    if idx >= total_motion:
        h = idx - total_motion
        if h <= zoom_in:
            f = 1 + (max_zoom-1)*(h/zoom_in)
        else:
            ho = h-zoom_in
            f = max_zoom - (max_zoom-1)*min(ho/zoom_out, 1)
    else:
        f = 1

    for i, (mn, mx) in enumerate(bounds):
        c = centers[i]; r = (mx-mn)/2
        low, high = c - r/f, c + r/f
        if i==0: ax.set_xlim(low, high)
        elif i==1: ax.set_ylim(low, high)
        else: ax.set_zlim(low, high)

    return scat, txt

#Build & save animation

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000/fps, blit=False)
writer = FFMpegWriter(fps=fps, metadata={'artist':'PragunNepal'}, bitrate=30000)
ani.save('HaloFormation_with_axes.mp4', writer=writer)
print('✅ Saved HaloFormation_with_axes.mp4')
```
---

## 🔭 <a name="conclusion"></a>4. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal